In [16]:
!pip install -U "tensorflow-text==2.13.*"
!pip install "tf-models-official==2.13.*"
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

ERROR: Could not find a version that satisfies the requirement tensorflow-text==2.13.* (from versions: 2.17.0rc0, 2.17.0)
ERROR: No matching distribution found for tensorflow-text==2.13.*
  Using cached tf_models_official-2.13.2-py2.py3-none-any.whl.metadata (1.4 kB)
INFO: pip is looking at multiple versions of tf-models-official to determine which version is compatible with other requirements. This could take a while.
  Using cached tf_models_official-2.13.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [37 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/t2/xw_g33n96rxfmq14n3dnpb6h0000gn/T/pip-install-_ndmcgd5/pyyaml_59600574cb734785a4400e0e7

In [2]:
import pandas as pd

df = pd.read_csv("top_and_non_golf_course_reviews.csv")
df.head()

,review_id,course_name,label,location,architect,year_built,review_title,review_author,file_name,review_text
0,1,Pine Valley,top100,"Pine Valley, NJ",George Crump / Harry S. Colt,1918,PINE VALLEY GOLF CLUB - 19 POINTS,David Jones,rev01_pine_valley_1,There’s not much point trying to do a hole-by-...
1,2,Pine Valley,top100,"Pine Valley, NJ",George Crump / Harry S. Colt,1918,"Pine Valley Golf Club (Clementon, New Jersey)",Bill Satterfield,rev02_pine_valley_2,What to Expect: Pine Valley is the finest gol...
2,3,Cypress Point,top100,"Pebble Beach, CA",Alister MacKenzie,1928,CYPRESS POINT REVIEW,Graylyn Loomis,rev03_cypress_point_1,“No one but a poet should be allowed to write ...
3,4,Cypress Point,top100,"Pebble Beach, CA",Alister MacKenzie,1928,"Cypress Point Golf Club (Pebble Beach, Califor...",Bill Satterfield,rev04_cypress_point_2,What to Expect: I don't even feel worthy to w...
4,5,Shinnecock Hills,top100,"Southampton, NY",William Flynn,1931,Review: Shinnecock Hills Golf Club,Andrew Harvie,rev05_shinnecock_1,"There’s not many courses as acclaimed, sought ..."


In [3]:
df.groupby('label').count()

,review_id,course_name,location,architect,year_built,review_title,review_author,file_name,review_text
label,,,,,,,,,
non100,60,60,60,60,60,60,60,60,60
top100,60,60,60,60,60,60,60,60,60


In [4]:
df.groupby('label').describe()

review_id                                                     \
           count  mean        std   min    25%   50%     75%    max   
label                                                                 
non100      60.0  90.5  17.464249  61.0  75.75  90.5  105.25  120.0   
top100      60.0  30.5  17.464249   1.0  15.75  30.5   45.25   60.0   

       year_built                                                          \
            count         mean        std     min     25%     50%     75%   
label                                                                       
non100       60.0  1989.666667  27.972545  1897.0  1989.0  1999.5  2003.0   
top100       60.0  1928.266667  26.483040  1895.0  1917.0  1924.5  1932.0   

                
           max  
label           
non100  2019.0  
top100  2003.0

In [5]:
df['top100'] = df['label'].apply(lambda x: 1 if x == 'top100' else 0)
df.head(3)
df.tail(3)

,review_id,course_name,label,location,architect,year_built,review_title,review_author,file_name,review_text,top100
117,118,Blackwolf Run,non100,"Kohler, WI",Pete Dye,1989,Golf Course Review: The River at Blackwolf Run,Derek Wise,rev118_blackwolf_run_2,The River at Blackwolf Run golf course has bee...,0
118,119,Black Creek Golf Club,non100,"Chattanooga, TN",Brian Silva,2000,"Black Creek Golf Club (Chattanooga, Tennessee)",Bill Satterfield,rev119_black_creek_club_1,What to Expect: Black Creek is a throwback in...,0
119,120,Black Creek Golf Club,non100,"Chattanooga, TN",Brian Silva,2000,BLACK CREEK CLUB,Phil Sokol,rev120_black_creek_club_2,HISTORY: When a course is only open for a ...,0


In [6]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [7]:
def get_review_embedding(reviews):
    preprocessed_text = bert_preprocess(reviews)
    return bert_encoder(preprocessed_text)['pooled_output']

In [8]:
# Example of how the get_review_embedding function works
get_review_embedding(["There’s not much point trying to do a hole-by", "No one but a poet should be allowed to write"])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.7295202 , -0.17719838,  0.1491849 , ...,  0.07682879,
        -0.56339663,  0.8735246 ],
       [-0.84179395, -0.44324878, -0.6954385 , ..., -0.34175068,
        -0.72697586,  0.7802301 ]], dtype=float32)>

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['review_text'], df['top100'], test_size=0.2, stratify=df['top100'], random_state=42)

In [10]:
X_train.head()

11    The Sandhills region of Nebraska is a huge mea...
77    Designed by Tom Fazio and opened in 1987, the ...
52    The California Golf Club of San Francisco date...
66    The Myrtle Beach area, known as the “Grand Str...
32    “One is a curiosity, two is a destination.” Mi...
Name: review_text, dtype: object

In [11]:
bert_preprocess = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/tensorFlow2/en-uncased-preprocess/3")
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/tensorFlow2/en-uncased-l-12-h-768-a-12/4")

In [12]:
def get_review_embedding(reviews):
    preprocessed_text = bert_preprocess(reviews)
    return bert_encoder(preprocessed_text)['pooled_output']

In [13]:
from tensorflow.keras.utils import plot_model

In [14]:
# Bert Layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output')(l)

# Construct the final model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

ValueError: Exception encountered when calling layer 'keras_layer_2' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_2' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=None, name=text>
  • training=None

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural Network Layers
l = tf.keras.layers.Dropout(0.1, name = 'dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output')(l)

# Use the inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])
